Group 12

Anakha Krishnavilasom Gopalakrishnan - 4 hours

Daniel Juster - 4 hours

# DAT405/DIT407 Introduction to Data Science and AI 
## 2022-2023, Reading Period 4
## Assignment 5: Reinforcement learning and classification

Hints:
You can execute certain linux shell commands by prefixing the command with `!`. You can insert Markdown cells and code cells. The first you can use for documenting and explaining your results the second you can use writing code snippets that execute the tasks required.  

This assignment is about **sequential decision making** under uncertainty (Reinforcement learning). In a sequential decision process, the process jumps between different states (the environment), and in each state the decision maker, or agent, chooses among a set of actions. Given the state and the chosen action, the process jumps to a new state. At each jump the decision maker receives a reward, and the objective is to find a sequence of decisions (or an optimal policy) that maximizes the accumulated rewards.

We will use **Markov decision processes** (MDPs) to model the environment, and below is a primer on the relevant background theory. 



* To make things concrete, we will first focus on decision making under **no** uncertainity (question 1 and 2), i.e, given we have a world model, we can calculate the exact and optimal actions to take in it. We will first introduce **Markov Decision Process (MDP)** as the world model. Then we give one algorithm (out of many) to solve it.

* (Optional) Next we will work through one type of reinforcement learning algorithm called Q-learning (question 3). Q-learning is an algorithm for making decisions under uncertainity, where uncertainity is over the possible world model (here MDP). It will find the optimal policy for the **unknown** MDP, assuming we do infinite exploration.

* Finally, in question 4 you will be asked to explain differences between reinforcement learning and supervised learning and in question 5 write about decision trees and random forests.

## Primer
### Decision Making
The problem of **decision making under uncertainty** (commonly known as **reinforcement learning**) can be broken down into
two parts. First, how do we learn about the world? This involves both the
problem of modeling our initial uncertainty about the world, and that of drawing conclusions from evidence and our initial belief. Secondly, given what we
currently know about the world, how should we decide what to do, taking into
account future events and observations that may change our conclusions?
Typically, this will involve creating long-term plans covering possible future
eventualities. That is, when planning under uncertainty, we also need to take
into account what possible future knowledge could be generated when implementing our plans. Intuitively, executing plans which involve trying out new
things should give more information, but it is hard to tell whether this information will be beneficial. The choice between doing something which is already
known to produce good results and experiment with something new is known
as the **exploration-exploitation dilemma**.

### The exploration-exploitation trade-off

Consider the problem of selecting a restaurant to go to during a vacation. Lets say the
best restaurant you have found so far was **Les Epinards**. The food there is
usually to your taste and satisfactory. However, a well-known recommendations
website suggests that **King’s Arm** is really good! It is tempting to try it out. But
there is a risk involved. It may turn out to be much worse than **Les Epinards**,
in which case you will regret going there. On the other hand, it could also be
much better. What should you do?
It all depends on how much information you have about either restaurant,
and how many more days you’ll stay in town. If this is your last day, then it’s
probably a better idea to go to **Les Epinards**, unless you are expecting **King’s
Arm** to be significantly better. However, if you are going to stay there longer,
trying out **King’s Arm** is a good bet. If you are lucky, you will be getting much
better food for the remaining time, while otherwise you will have missed only
one good meal out of many, making the potential risk quite small.

### Markov Decision Processes
Markov Decision Processes (MDPs) provide a mathematical framework for modeling sequential decision making under uncertainty. An *agent* moves between *states* in a *state space* choosing *actions* that affects the transition probabilities between states, and the subsequent *rewards* recieved after a jump. This is then repeated a finite or infinite number of epochs. The objective, or the *solution* of the MDP, is to optimize the accumulated rewards of the process.

Thus, an MDP consists of five parts: 

* Decision epochs: $t={1,2,...,T}$, where $T\leq \infty$
* State space: $S=\{s_1,s_2,...,s_N\}$ of the underlying environment
* Action space $A=\{a_1,a_2,...,a_K\}$ available to the decision maker at each decision epoch
* Transition probabilities $p(s_{t+1}|s_t,a_t)$ for jumping from state $s_t$ to state $s_{t+1}$ after taking action $a_t$
* Reward functions $R_t = r(a_t,s_t,s_{t+1})$ resulting from the chosen action and subsequent transition

A *decision policy* is a function $\pi: s \rightarrow a$, that gives instructions on what action to choose in each state. A policy can either be *deterministic*, meaning that the action is given for each state, or *randomized* meaning that there is a probability distribution over the set of possible actions for each state. Given a specific policy $\pi$ we can then compute the the *expected total reward* when starting in a given state $s_1 \in S$, which is also known as the *value* for that state, 

$$V^\pi (s_1) = E\left[ \sum_{t=1}^{T} r(s_t,a_t,s_{t+1}) {\Large |} s_1\right] = \sum_{t=1}^{T} r(s_t,a_t,s_{t+1}) p(s_{t+1} | a_t,s_t)$$ 

where $a_t = \pi(s_t)$. To ensure convergence and to control how much credit to give to future rewards, it is common to introduce a *discount factor* $\gamma \in [0,1]$. For instance, if we think all future rewards should count equally, we would use $\gamma = 1$, while if we value near-future rewards higher than more distant rewards, we would use $\gamma < 1$. The expected total *discounted* reward then becomes

$$V^\pi( s_1) = \sum_{t=1}^T \gamma^{t-1} r(s_t,a_t, s_{t+1}) p(s_{t+1} | s_t, a_t) $$

Now, to find the *optimal* policy we want to find the policy $\pi^*$ that gives the highest total reward $V^*(s)$ for all $s\in S$. That is, we want to find the policy where

$$V^*(s) \geq V^\pi(s), s\in S$$

To solve this we use a dynamic programming equation called the *Bellman equation*, given by

$$V(s) = \max_{a\in A} \left\{\sum_{s'\in S} p(s'|s,a)( r(s,a,s') +\gamma V(s')) \right\}$$

It can be shown that if $\pi$ is a policy such that $V^\pi$ fulfills the Bellman equation, then $\pi$ is an optimal policy.

A real world example would be an inventory control system. The states could be the amount of items we have in stock, and the actions would be the amount of items to order at the end of each month. The discrete time would be each month and the reward would be the profit. 


## Question 1

The first question covers a deterministic MPD, where the action is directly given by the state, described as follows:

* The agent starts in state **S** (see table below)
* The actions possible are **N** (north), **S** (south), **E** (east), and **W** west. 
* The transition probabilities in each box are deterministic (for example P(s'|s,N)=1 if s' north of s). Note, however, that you cannot move outside the grid, thus all actions are not available in every box.
* When reaching **F**, the game ends (absorbing state).
* The numbers in the boxes represent the rewards you receive when moving into that box. 
* Assume no discount in this model: $\gamma = 1$
    
    
| | | |
|----------|----------|---------|
|-1 |1|**F**|
|0|-1|1|  
|-1 |0|-1|  
|**S**|-1|1|

Let $(x,y)$ denote the position in the grid, such that $S=(0,0)$ and $F=(2,3)$.


**1a)** What is the optimal path of the MDP above? Is it unique? Submit the path as a single string of directions. E.g. NESW will make a circle.

Answer 1a:

we see that we have 5 "-1", 3 "+1" and 2 "0", so it will be hard to get a high score. In fact, after checking options (below) we see that 

EENNN = 0

EENNWNE = 0

are the highest scores we can get. They are not unique.

**Full list:**


E N E N N = -1 

E N E N W N E = -1

E N E N WW N EE = -2

E NN W N EE = -2 

E N W N EE N = -2

E N W N E N E = -2

E N W NN E S E N = -2


N EE NN = -1

N E N E N = - 1

N E NN E = -1

NN EE N = -1 

NN E N E = -1 

NNN E S E N = -1 

NNN EE = - 1


EE NN N = 0

EE NN W N E = 0

EE NN WW N E E = -1

EE N W N E N =  -1

EE N W NN E = -1

EE N WW N EE N = -2

EE N WW NN E E = -2

EE N WW NE N E = -2

EE N WW NN E E = -2

EE N WW NN E S E N = -2










**1b)** What is the optimal policy (i.e. the optimal action in each state)? It is helpful if you draw the arrows/letters in the grid.

Answer 1b:

Optimal action from each position in the matrix is the best value we can get. It should be 0, 1 or -1. E.g it is only possible to go from S (position (0,0) to 2 positions: N (1,0) or E (0,1). They both give the same reward: -1


This is the full list:


Position->Optimal action->score

(0,0)->	N/E->	-1

(0,1)->	E->	1

(0,2)->	N/W->	-1

(1,0)->	N/E->	0

(1,1)->	N/E/S/W->	-1

(1,2)->	N/S->	1

(2,0)->	N/E/S->	-1

(2,1)->	N/E->	1

(2,2)->	-> N	0 (going to F)

(3,0)->	E	-> 1

(3,1)->	E	-> 0 (going to F)

(3,2)->	W/S -> 1 



**1c)** What is expected total reward for the policy in 1a)?




Answer 1c:

The expected total reward is 0.

## Value Iteration

For larger problems we need to utilize algorithms to determine the optimal policy $\pi^*$. *Value iteration* is one such algorithm that iteratively computes the value for each state. Recall that for a policy to be optimal, it must satisfy the Bellman equation above, meaning that plugging in a given candidate $V^*$ in the right-hand side (RHS) of the Bellman equation should result in the same $V^*$ on the left-hand side (LHS). This property will form the basis of our algorithm. Essentially, it can be shown that repeated application of the RHS to any intial value function $V^0(s)$ will eventually lead to the value $V$ which statifies the Bellman equation. Hence repeated application of the Bellman equation will also lead to the optimal value function. We can then extract the optimal policy by simply noting what actions that satisfy the equation.    

The process of repeated application of the Bellman equation is what we here call the _value iteration_ algorithm. It practically procedes as follows:

```
epsilon is a small value, threshold
for x from i to infinity 
do
    for each state s
    do
        V_k[s] = max_a Σ_s' p(s′|s,a)*(r(a,s,s′) + γ*V_k−1[s′])
    end
    if  |V_k[s]-V_k-1[s]| < epsilon for all s
        for each state s,
        do
            π(s)=argmax_a ∑_s′ p(s′|s,a)*(r(a,s,s′) + γ*V_k−1[s′])
            return π, V_k 
        end
end

```

**Example:** We will illustrate the value iteration algorithm by going through two iterations. Below is a 3x3 grid with the rewards given in each state. Assume now that given a certain state $s$ and action $a$, there is a probability 0.8 that that action will be performed and a probability 0.2 that no action is taken. For instance, if we take action **E** in state $(x,y)$ we will go to $(x+1,y)$ 80 percent of the time (given that that action is available in that state), and remain still 20 percent of the time. We will use have a discount factor $\gamma = 0.9$. Let the initial value be $V^0(s)=0$ for all states $s\in S$. 

**Reward**:

| | | |  
|----------|----------|---------|  
|0|0|0|
|0|10|0|  
|0|0|0|  


**Iteration 1**: The first iteration is trivial, $V^1(s)$ becomes the $\max_a \sum_{s'} p(s'|s,a) r(s,a,s')$ since $V^0$ was zero for all $s'$. The updated values for each state become

| | | |  
|----------|----------|---------|  
|0|8|0|
|8|2|8|  
|0|8|0|  
  
**Iteration 2**:  
  
Staring with cell (0,0) (lower left corner): We find the expected value of each move:  
Action **S**: 0  
Action **E**: 0.8( 0 + 0.9 \* 8) + 0.2(0 + 0.9 \* 0) = 5.76  
Action **N**: 0.8( 0 + 0.9 \* 8) + 0.2(0 + 0.9 \* 0) = 5.76  
Action **W**: 0

Hence any action between **E** and **N** would be best at this stage.

Similarly for cell (1,0):

Action **N**: 0.8( 10 + 0.9 \* 2) + 0.2(0 + 0.9 \* 8) = 10.88 (Action **N** is the maximizing action)  

Similar calculations for remaining cells give us:

| | | |  
|----------|----------|---------|  
|5.76|10.88|5.76|
|10.88|8.12|10.88|  
|5.76|10.88|5.76|  


## Question 2

**2a)** Code the value iteration algorithm just described here, and show the converging optimal value function and the optimal policy for the above 3x3 grid. Make sure to consider that there may be several equally good actions for a state when presenting the optimal policy.

In [23]:
# Answer 2a


# Import library
import numpy as np

# Input values
# ============
# Number of iterations the algorithm should make
iterations = 0 

# Discount factor 𝛾=0.9
gamma = 0.9 

# Probability that an action is taken
actionTaken = 0.8 

# Probability that NO action is taken
actionNotTaken = 0.2 

# When to stop iterate. epsilon is a small value = threshold
epsilon = 0.001


# Start values
# ============
# Rewards
rewards = np.array([[0,0,0],[0,10,0],[0,0,0]]) 

# States
states = np.array([[0.0,0.0,0.0],[0.0,0.0,0.0],[0.0,0.0,0.0]]) 

# New states
new_states = np.array([[0.0,0.0,0.0],[0.0,0.0,0.0],[0.0,0.0,0.0]]) 

# Function for computing values
def computeValue(currentReward, nextReward, currentStateValue, nextStateValue, gammaValue, actionTaken):
    v = actionTaken*(nextReward+gammaValue*nextStateValue)+(1-actionTaken)*(currentReward+gammaValue*currentStateValue)
    return v

# Iteratate
while True:

    # north, east, west, south = computed V values for each move
    
    for row in range(0,states.shape[0]):
        
        for column in range(states.shape[1]):
            
            # Start with 0
            north, east, west, south = 0.0,0.0,0.0,0.0
            
            # N border check
            if row != 0:
                # North value
                north = computeValue(rewards[row,column], rewards[row-1,column], states[row,column], states[row-1,column], gamma, actionTaken)

            #E border check
            if column != (rewards.shape[0] - 1):
                # East value
                east = computeValue(rewards[row,column], rewards[row,column+1], states[row,column], states[row,column+1], gamma, actionTaken)
            
            #W border check
            if column != 0:
                # West value
                west = computeValue(rewards[row,column], rewards[row,column-1], states[row,column], states[row,column-1], gamma, actionTaken)
                
            #S border check
            if row != (rewards.shape[1] - 1):
                # South value
                south = computeValue(rewards[row,column], rewards[row+1,column], states[row,column], states[row+1,column], gamma, actionTaken)
    
            # Return max value
            new_states[row,column] = max(north, east, south, west)
    
    # Current difference
    threshold_state = np.absolute(states - new_states)
    
    # Threshold check 
    is_under_threshold = np.all((threshold_state < epsilon))
    
    # Save states 
    np.copyto(states, new_states)

    # Keep track of iterations
    iterations+=1 
    
    # Threshold
    if is_under_threshold:
        print('Converging optimal value = '+str(iterations)+'\n', '--------------------')
        break

print('Optimal policy values =\n')
np.set_printoptions(precision=2)

# Time to print resulting matrix
print(states)





Converging optimal value = 82
 --------------------
Optimal policy values =

[[45.6  51.94 45.6 ]
 [51.94 48.04 51.94]
 [45.6  51.94 45.6 ]]


**2b)** Explain why the result of 2a) does not depend on the initial value $V_0$.

Answer 2b:


We are looking for values dependent on the gamma-factor (0.9) and the probablilty (0.8 and 0.2) when a threshold has been met. The inital value for V will only impact how many iterations we need.


**2c)** Describe your interpretation of the discount factor $\gamma$. What would happen in the two extreme cases $\gamma = 0$ and $\gamma = 1$? Given some MDP, what would be important things to consider when deciding on which value of $\gamma$ to use?

Answer 2c:

discount factor 𝛾 is used to indicate if a direct reward (next action) is more important than a future reward. So a discount an future rewards might mean that they eventually becomes unimportant. If 𝛾 = 0, this means that only the first reward is valuable. All other will have 0 value. And if 𝛾 = 1, all actions will generate the same reward. 

If we have an MDP with many states and we want it to run for a long time, we might want to consider a high discount factor. At the same time, if we want to stress that immediate rewards are important, a low discount factor is better.

## Reinforcement Learning (RL) (Theory for optional question 3)
Until now, we understood that knowing the MDP, specifically $p(s'|a,s)$ and $r(s,a,s')$ allows us to efficiently find the optimal policy using the value iteration algorithm. Reinforcement learning (RL) or decision making under uncertainity, however, arises from the question of making optimal decisions without knowing the true world model (the MDP in this case).

So far we have defined the value function for a policy through $V^\pi$. Let's now define the *action-value function*

$$Q^\pi(s,a) = \sum_{s'} p(s'|a,s) [r(s,a,s') + \gamma V^\pi(s')]$$

The value function and the action-value function are directly related through

$$V^\pi (s) = \max_a Q^\pi (s,a)$$

i.e, the value of taking action $a$ in state $s$ and then following the policy $\pi$ onwards. Similarly to the value function, the optimal $Q$-value equation is:

$$Q^*(s,a) = \sum_{s'} p(s'|a,s) [r(s,a,s') + \gamma V^*(s')]$$

and the relationship between $Q^*(s,a)$ and $V^*(s)$ is simply

$$V^*(s) = \max_{a\in A} Q^*(s,a).$$

#### Q-learning

Q-learning is a RL-method where the agent learns about its unknown environment (i.e. the MDP is unknown) through exploration. In each time step *t* the agent chooses an action *a* based on the current state *s*, observes the reward *r* and the next state *s'*, and repeats the process in the new state. Q-learning is then a method that allows the agent to act optimally. Here we will focus on the simplest form of Q-learning algorithms, which can be applied when all states are known to the agent, and the state and action spaces are reasonably small. This simple algorithm uses a table of Q-values for each $(s,a)$ pair, which is then updated in each time step using the update rule in step $k+1$

$$Q_{k+1}(s,a) = Q_k(s,a) + \alpha \left( r(s,a) + \gamma \max \{Q_k(s',a')\} - Q_k(s,a) \right) $$ 

where $\gamma$ is the discount factor as before, and $\alpha$ is a pre-set learning rate. It can be shown that this algorithm converges to the optimal policy of the underlying MDP for certain values of $\alpha$ as long as there  is sufficient exploration. For our case, we set a constant $\alpha=0.1$.

#### OpenAI Gym

We shall use already available simulators for different environments (worlds) using the popular [OpenAI Gym library](https://www.gymlibrary.dev/). It just implements different types of simulators including ATARI games. Although here we will only focus on simple ones, such as the **Chain enviroment** illustrated below.
![alt text](https://chalmersuniversity.box.com/shared/static/6tthbzhpofq9gzlowhr3w8if0xvyxb2b.jpg)
The figure corresponds to an MDP with 5 states $S = \{1,2,3,4,5\}$ and two possible actions $A=\{a,b\}$ in each state. The arrows indicate the resulting transitions for each state-action pair, and the numbers correspond to the rewards for each transition.

## Question 3 (optional)
You are to first familiarize with the framework of [the OpenAI environments](https://www.gymlibrary.dev/), and then implement the Q-learning algorithm for the <code>NChain-v0</code> enviroment depicted above, using default parameters and a learning rate of $\gamma=0.95$. Report the final $Q^*$ table after convergence of the algorithm. For an example on how to do this, you can refer to the Q-learning of the **Frozen lake environment** (<code>q_learning_frozen_lake.ipynb</code>), uploaded on Canvas. Hint: start with a small learning rate.

Note that the NChain environment is not available among the standard environments, you need to load the <code>gym_toytext</code> package, in addition to the standard gym:

<code>
!pip install gym-legacy-toytext<br>
import gym<br>
import gym_toytext<br>
env = gym.make("NChain-v0")<br>
</code>

In [24]:
# Answer 3

# Tried to run the Frozen lake environment, but it did not work. The render-function failed.

## Question 4

**4a)** What is the importance of exploration in reinforcement learning? Explain with an example.

Answer 4a:

From the lecture we learnt that "explore means taking a random action and to exploit means taking whatever action seems most rewarding at the present moment".
So, by exploring we will learn how to find optimal solutions and by only exploiting, short term gains might be gained, but the long term rewards might get lost. E.g. if you always eat the same food, you know that you will get your meal, but by exploring (exploration)  different kind of food, you might get more nutrition and better taste. A combination is good.

**4b)** Explain what makes reinforcement learning different from supervised learning tasks such as regression or classification. 

Answer 4b:


Reinformcement learning is a way to do "trail and error" until an optimal solution is found. This can take some time....
Examples where reinformcement learning can be good is autonomous driving and recommendation engines. They will learn over time and become better.

Supervised learning is used when we have testdate that can be used as a basis to classify things, like spam or face recognition. 

## Question 5

**5a)** Give a summary of how a decision tree works and how it extends to random forests.

Answer 5a:


Decision trees are basically a structure of classification questions (branches) that leads to results (leaves) making up a "tree". This can be used to classify animals or finding out what the underlying reason for a patients condition thus leading to a suggested treatment.

The trick here is to construct a tree out of questions that identify the root cause. In what order shall we ask the questions. If we are investigating for a heart condition, should we start by asking for chest pain, check the blood-fat or for blocked arteries? For this we can investigate a focus group and determine probablities for all questions, and generate an impurity index, e.g Gini impurity (or perhaps Shannon’s notion of entropy). Lowest Gini-value will be the best to start with and this will be the root. 

So, decision trees are easy to understand and create, but they can be big and are not always accurate. In fact they have a high level of inaccuracy.

This is where "random forest" comes in. Random forest is multiple decision trees, built from the same datasets, but with bootstrapped datasets (randomly picked lines of data). We do this over and over again, creating a forest. The resulting accuracy is higher than for decision trees.






**5b)** State at least one advantage and one drawback with using random forests over decision trees.

Answer 5b:


**Random forests advantage**

highly accurate

robust method

**Random forests drawback**

Slow (need to use all trees)

Not so easy to interpret


# References
Primer/text based on the following references:
* http://www.cse.chalmers.se/~chrdimi/downloads/book.pdf
* https://github.com/olethrosdc/ml-society-science/blob/master/notes.pdf